In [3]:
import tensorflow as tf
import re
import numpy as np
import os
import time
import pandas as pd

In [4]:
with open('abstracts.txt', 'r') as abstracts_file:
     abstracts = abstracts_file.read() 


In [14]:
vocab = sorted(set(abstracts))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in abstracts])

In [15]:
print('{')
for char,_ in zip(char2idx, range(35)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  '3' :  20,
  '4' :  21,
  '5' :  22,
  '6' :  23,
  '7' :  24,
  '8' :  25,
  '9' :  26,
  ':' :  27,
  ';' :  28,
  '<' :  29,
  '=' :  30,
  '>' :  31,
  '?' :  32,
  '@' :  33,
  'A' :  34,
  ...
}


In [16]:
seq_length = 100
examples_per_epoch = len(abstracts)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
b
s
t
r


In [18]:
seq_length = 100
examples_per_epoch = len(abstracts)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
BATCH_SIZE = 128

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [20]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 193) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 128)          24704     
_________________________________________________________________
gru (GRU)                    (128, None, 512)          986112    
_________________________________________________________________
dense (Dense)                (128, None, 193)          99009     
Total params: 1,109,825
Trainable params: 1,109,825
Non-trainable params: 0
_________________________________________________________________


In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)

In [24]:
model.compile(optimizer='adam', loss=loss)


In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=5

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 2200 steps
Epoch 1/5
2200/2200 [==============================] - 2092s 951ms/step - loss: 1.1943
Epoch 2/5
2200/2200 [==============================] - 2078s 945ms/step - loss: 1.1516
Epoch 3/5
2200/2200 [==============================] - 2311s 1s/step - loss: 1.1296
Epoch 4/5
2200/2200 [==============================] - 2082s 946ms/step - loss: 1.1159
Epoch 5/5
2200/2200 [==============================] - 2072s 942ms/step - loss: 1.1063


In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            24704     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 512)            986112    
_________________________________________________________________
dense_1 (Dense)              (1, None, 193)            99009     
Total params: 1,109,825
Trainable params: 1,109,825
Non-trainable params: 0
_________________________________________________________________


In [48]:
def generate_text(model, start_string):
  
    num_generate = 500

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    temperature = .5

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [49]:
print(generate_text(model, start_string=u"Abstact\n\nThe corona"))

Abstact

The coronavirus (HCoV) is a common cause of transmission of the protein (MS) is a result of the first to detect and recognize a three-dimensional respiratory tract of the antiviral drugs, but not more common cancer in the possibility of the ability to identify replication of the envelope protein of the human coronavirus (SARS-CoV) is an important role in the transmission of the host inntation. The potential of the viral envelope and in vivo events in the first the infection control in the serology of the 
